In [ ]:
pip install -q datasets trl huggingface_hub[hf_transfer] python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
from datasets import features, load_dataset
from transformers import AutoModelForVision2Seq, AutoProcessor
import torch
from trl import DPOConfig, DPOTrainer
from peft import LoraConfig
from dotenv import load_dotenv
from huggingface_hub import login
import wandb

load_dotenv()  # This loads the variables from the .env file
wandb_api_key = os.getenv("WANDB_API_KEY")
hf_token = os.getenv("HF_TOKEN")

if not wandb_api_key or not hf_token:
    raise ValueError("WANDB_API_KEY or HF_TOKEN is not set in environment variables.")

wandb.login(key=wandb_api_key)

In [ ]:
# Load the model and processor
model = AutoModelForVision2Seq.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.bfloat16, device_map="auto")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", do_image_splitting=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


In [ ]:
# Load the dataset
dataset = load_dataset("abshetty/llava-chains-10k-shortlist-individual", split="train")

README.md:   0%|          | 0.00/800 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7714 [00:00<?, ? examples/s]

In [ ]:
def format(example):
    # Prepare the input for the chat template
    prompt = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": example["question_full"]}]}]
    chosen = [{"role": "assistant", "content": [{"type": "text", "text": example["best_response"]}]}]
    rejected = [{"role": "assistant", "content": [{"type": "text", "text": example["worst_response"]}]}]
    # Apply the chat template
    prompt = processor.apply_chat_template(prompt, tokenize=False)
    chosen = processor.apply_chat_template(chosen, tokenize=False)
    rejected = processor.apply_chat_template(rejected, tokenize=False)
    # Resize the image to ensure it fits within the maximum allowable
    # size of the processor to prevent OOM errors.
    max_size = processor.image_processor.size["shortest_edge"] // 2
    example["image"].thumbnail((max_size, max_size))
    return {"images": [example["image"]], "prompt": prompt, "chosen": chosen, "rejected": rejected}

# Apply the formatting function to the dataset
dataset = dataset.map(format, remove_columns=dataset.column_names, num_proc=32)

# Make sure that the images are decoded, it prevents from storing bytes.
f = dataset.features
f["images"] = features.Sequence(features.Image(decode=True))
dataset = dataset.cast(f)


Map (num_proc=32):   0%|          | 0/7714 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7714 [00:00<?, ? examples/s]

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)

# Extract the train and eval sets
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
# Train the model
training_args = DPOConfig(
    output_dir="lora-12-07-individual-shortlist",
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps=1,
    learning_rate=1e-5,
    beta=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    num_train_epochs=2,
 #   weight_decay=0.01,
 #   rpo_alpha=0.1,
    dataset_num_proc=32,  # tokenization will use 32 processes
    dataloader_num_workers=32,  # data loading will use 32 workers
    logging_steps=1,
)

# Define LoRA configuration with specified rank
lora_config = LoraConfig(
    r=64,  # Set rank to 64
    lora_alpha=128,  # Set scaling factor to 128
    target_modules="all-linear",  # Target all linear layers
    lora_dropout=0.1,
)

trainer = DPOTrainer(
    model,
    ref_model=None,  # not needed when using peft
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    peft_config=lora_config,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Extracting prompt from train dataset (num_proc=32):   0%|          | 0/6942 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=32):   0%|          | 0/6942 [00:00<?, ? examples/s]

Extracting prompt from eval dataset (num_proc=32):   0%|          | 0/772 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=32):   0%|          | 0/772 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=32):   0%|          | 0/6942 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=32):   0%|          | 0/772 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,0.693100,0.693147,0.000000,0.000000,0.000000,0.000000,-1337.952759,-1247.005859,-2.609573,-2.600135
2,0.693100,0.692632,0.000046,-0.001741,0.523316,0.001788,-1337.952393,-1247.023315,-2.612061,-2.602998
3,0.690800,0.678249,0.257932,0.222591,0.611399,0.035341,-1335.373413,-1244.779907,-2.613235,-2.603821
4,0.686700,0.656718,0.663845,0.562177,0.634715,0.101668,-1331.314331,-1241.384033,-2.614352,-2.604514
5,0.668100,0.628454,0.956906,0.766222,0.678756,0.190684,-1328.383789,-1239.343628,-2.587174,-2.576620
6,0.644500,0.590625,1.068806,0.771549,0.709845,0.297257,-1327.264771,-1239.290283,-2.561623,-2.550348
7,0.628900,0.566676,0.589182,0.197529,0.727979,0.391653,-1332.060913,-1245.030640,-2.523239,-2.509046
8,0.571500,0.558110,0.646279,0.098082,0.718912,0.548197,-1331.489990,-1246.024902,-2.476751,-2.460419
9,0.600300,0.531329,1.607023,0.861987,0.735751,0.745035,-1321.882568,-1238.385986,-2.448640,-2.431732
10,0.451000,0.518977,2.738961,1.784405,0.751295,0.954556,-1310.563232,-1229.161865,-2.419657,-2.402730


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=54, training_loss=0.506686606892833, metrics={'train_runtime': 25633.4445, 'train_samples_per_second': 0.542, 'train_steps_per_second': 0.002, 'total_flos': 0.0, 'train_loss': 0.506686606892833, 'epoch': 1.9907834101382489})

In [ ]:
# Saving the fine-tuned model and tokenizer
trainer.model.save_pretrained("final_checkpoint")
processor.save_pretrained("final_checkpoint")

['final_checkpoint/processor_config.json']

In [ ]:
trainer.push_to_hub("abshetty/llava-lora-individual-shortlist")



/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


  0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

(…)t.tfevents.1733618094.ce287168e38c.613.0:   0%|          | 0.00/86.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.07k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abshetty/lora-12-07-individual-shortlist/commit/8c8dfe7614c5c589e6d950081a8c8d3c8187ee93', commit_message='abshetty/llava-lora-individual-shortlist', commit_description='', oid='8c8dfe7614c5c589e6d950081a8c8d3c8187ee93', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abshetty/lora-12-07-individual-shortlist', endpoint='https://huggingface.co', repo_type='model', repo_id='abshetty/lora-12-07-individual-shortlist'), pr_revision=None, pr_num=None)